<a href="https://colab.research.google.com/github/jl3720/place_cell_modelling/blob/main/notebooks/SampledTestwColorJitters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mp
import tensorflow as tf
import math, os, glob, re
from sklearn.preprocessing import OneHotEncoder 
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
ss = StandardScaler() 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
#from numpy.core.fromnumeric import argmax
from sklearn.model_selection import train_test_split



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/A Project/Data/Images'
images = pd.Series(glob.glob(os.path.join(path,"*/*.png"),recursive=True),name= 'image')


In [ ]:
test_series = images[::5]
images.drop(index = [i for i in range(0,len(images),5)])

1        /content/drive/MyDrive/A Project/Data/Images/P...
2        /content/drive/MyDrive/A Project/Data/Images/P...
3        /content/drive/MyDrive/A Project/Data/Images/P...
4        /content/drive/MyDrive/A Project/Data/Images/P...
6        /content/drive/MyDrive/A Project/Data/Images/P...
                               ...                        
11112    /content/drive/MyDrive/A Project/Data/Images/R...
11113    /content/drive/MyDrive/A Project/Data/Images/R...
11114    /content/drive/MyDrive/A Project/Data/Images/R...
11116    /content/drive/MyDrive/A Project/Data/Images/R...
11117    /content/drive/MyDrive/A Project/Data/Images/R...
Name: image, Length: 8894, dtype: object

###I used a block to get these values, I'm going to try running it with the actual person and see what max and min values I can get

ymin: -1695

ymax: -220

xmin: 200

xmax: 2610



#Preprocessing


###Creating Dataframe with imagepaths in one column and the normalized x,y and theta values

In [ ]:
def get_dataframe(series):
  


  # images = glob.glob(os.path.join(path,"*/*.png"),recursive=True)
  # images_series = pd.Series(images,name = 'image')
  # #x_train = np.zeros((len(images),3))

  df = pd.DataFrame({'image':series})

  def get_coordinates(filename):
    xmax = 2610
    xmin = 200
    ymax = 1695
    ymin = 220
    match = re.findall(r"(X|Y)=(-?\d+\.\d+)", filename) 
    theta, x, y = [float(i[-1]) for i in match]

    #Scaling x,y and theta to be between 0 and 1
    if theta < 0:
      theta +=360
    theta /=360
    # if x> xmax or x <xmin:
    #   print("x = ",x)
    x = (x-xmin)/(xmax-xmin)
    y*= -1
    # if y>ymax or y<ymin:
    #   print("y =",y)
    y = (y -ymin)/(ymax-ymin)
    return np.array([x,y,theta])

  x_train = df['image'].apply(lambda x: pd.Series(get_coordinates(x)))
  df = df.join(x_train)
  df = df.rename(columns={0: 'x', 1: 'y', 2: 'theta'})
  return df

  

In [ ]:
train_df = get_dataframe(images)
test_df = get_dataframe(test_series)

###Creating dataframe

Check for correct scaling

In [ ]:
max(train_df['x']),min(train_df['x'])


(1.106795020746888, -0.01521535269709544)

#####Set picture dimensions here

In [ ]:
dim1 = 128
dim2 = 128

In [ ]:
def color_jitter_and_zoom(image):
  # Randomly adjust brightness, contrast, saturation, and hue
  image = tf.image.random_brightness(image, max_delta=0.3)
  image = tf.image.random_contrast(image, lower=0.2, upper=1.8)
  image = tf.image.random_saturation(image, lower=0.2, upper=1.8)
  image = tf.image.random_hue(image, max_delta=0.3)

  #Adjusting Zoom
  image = tf.keras.preprocessing.image.random_zoom(image, zoom_range=(0.9, 1.1))
  return image



we want to get about 10,000

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    #validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    preprocessing_function=color_jitter_and_zoom
  
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


In [ ]:
def validate_file(filepath):
    if os.path.isfile(filepath) and ".png" in filepath:
        return filepath
    else:
        return None

train_df['image'] = train_df['image'].apply(validate_file)
train_df.dropna(subset=['image'], inplace=True)

In [ ]:
test_df['image'] = test_df['image'].apply(validate_file)
test_df.dropna(subset=['image'], inplace=True)

In [ ]:
bs = 128

In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='image',
    y_col=['x','y','theta'],
    target_size=(dim1, dim2),
    color_mode='rgb',
    class_mode='raw',
    batch_size=bs,
    shuffle=True,
    seed=42)
    #subset='training',
    #validate_filenames = False)


# val_images = train_generator.flow_from_dataframe(
#     dataframe=train_df,
#     x_col='image',
#     y_col=['x','y','theta'],
#     target_size=(dim1, dim2),
#     color_mode='rgb',
#     class_mode='raw',
#     batch_size=32,
#     shuffle=True,
#     seed=42,
#     subset='validation',
#     #validate_filenames = False)


test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='image',
    y_col=['x','y','theta'],
    target_size=(dim1, dim2),
    color_mode='rgb',
    class_mode='raw',
    batch_size=bs,
    shuffle=False)
    #validate_filenames = False)


Found 11118 validated image filenames.
Found 2224 validated image filenames.


We can split train/validation/test by leaving out paths. 

#CNN Architecture

In [ ]:
def make_model(dim1,dim2):

  model = tf.keras.models.Sequential()
  # Input layer
  model.add(tf.keras.layers.Input(shape=(dim1, dim2, 3)))

  # Convolutional layer 1
  model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'))

  # Pooling layer 1
  model.add(tf.keras.layers.MaxPooling2D((2, 2)))
  
  #Dropout Layer 
  model.add(tf.keras.layers.Dropout(0.4)) 

  # Convolutional layer 2
  model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'))

  # Pooling layer 2
  model.add(tf.keras.layers.MaxPooling2D((2, 2)))

  #Dropout Layer 
  model.add(tf.keras.layers.Dropout(0.4)) 

  # Convolutional layer 3
  model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
  
  # Pooling layer 3
  model.add(tf.keras.layers.MaxPooling2D((2, 2)))

  #Dropout Layer 
  model.add(tf.keras.layers.Dropout(0.4)) 

  # Convolutional layer 4
  model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'))

  # Pooling layer 4
  model.add(tf.keras.layers.MaxPooling2D((2, 2)))

  #Dropout Layer 
  model.add(tf.keras.layers.Dropout(0.4)) 

  # Flatten the feature maps
  model.add(tf.keras.layers.Flatten())
  
  # 1-2 Flat Layers
  model.add(tf.keras.layers.Dense(256, activation='relu'))

  #model.add(tf.keras.layers.Dropout(0.25)) 

  model.add(tf.keras.layers.Dense(128, activation='relu'))

  #model.add(tf.keras.layers.Dropout(0.25)) 

  #Place Cell Behaviour
  model.add(tf.keras.layers.Dense(32, activation='linear'))

  #Decoding
  model.add(tf.keras.layers.Dense(16, activation='relu'))

  model.add(tf.keras.layers.Dense(8, activation='relu'))

  model.add(tf.keras.layers.Dense(3,activation= 'linear'))

  # Compile the model
  model.compile(optimizer='adam', loss='mse')  #,metrics=[tf.keras.metrics.RootMeanSquaredError()])

  return model

## WE can halve the numbers

In [ ]:
cnn = make_model(dim1,dim2)
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 64, 64, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 32, 32, 32)        0

In [ ]:
history = cnn.fit(
    train_images,,
    validation_data=test_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100
87/87 [==============================] - 8774s 101s/step - loss: 0.1126 - val_loss: 0.1035
Epoch 2/100
87/87 [==============================] - 1077s 12s/step - loss: 0.0858 - val_loss: 0.1076
Epoch 3/100
87/87 [==============================] - 1068s 12s/step - loss: 0.0815 - val_loss: 0.1010
Epoch 4/100
87/87 [==============================] - 1082s 12s/step - loss: 0.0796 - val_loss: 0.0990
Epoch 5/100
87/87 [==============================] - 1056s 12s/step - loss: 0.0788 - val_loss: 0.1029
Epoch 6/100
87/87 [==============================] - 1066s 12s/step - loss: 0.0778 - val_loss: 0.1054
Epoch 7/100
87/87 [==============================] - 1065s 12s/step - loss: 0.0766 - val_loss: 0.0985
Epoch 8/100
87/87 [==============================] - 1075s 12s/step - loss: 0.0761 - val_loss: 0.1017
Epoch 9/100
87/87 [==============================] - 1055s 12s/step - loss: 0.0764 - val_loss: 0.1014
Epoch 10/100
87/87 [==============================] - 1089s 13s/step - loss: 0.07

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.show()


In [ ]:
cnn.save('/content/drive/MyDrive/A Project/CNN Stuff]'\/cnn_what_a_great_start')

In [ ]:
true = test_images.labels
pred = cnn.predict(test_images)

Look into bootstrapping and check vs. random (completely random)

In [ ]:
from sklearn.metrics import mean_squared_error as mse
mse_x = mse(true[:,0],pred[:,0])
mse_y = mse(true[:,1],pred[:,1])
mse_theta = mse(true[:,2],pred[:,2])
amse = (mse_x + mse_y + mse_theta)/3
rmse = np.sqrt(amse)
print(mse_x,mse_y,mse_theta,sep = '\n')
amse,rmse

one graph of absolute distance error 

one graph of angle error 

do those graphs for each type (CNN vs. random vs. bootstrapping)

In [ ]:
plt.plot(true[:,0])
plt.plot(pred[:,0])

In [ ]:
plt.plot(true[:,1])
plt.plot(pred[:,1])


In [ ]:
plt.plot(true[:,2])
plt.plot(pred[:,2])